In [1]:
# --- 0. CONNECT TO THE QUANTUM LOOM ---
import warnings
import os
import logging
import sys

# 1. Kill Standard Warnings
warnings.simplefilter("ignore")
warnings.filterwarnings("ignore")

# 2. Kill ResourceWarnings (Socket warnings from Qiskit Runtime)
warnings.filterwarnings("ignore", category=ResourceWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

# 3. Kill System Logs (TensorFlow/Qiskit/Urllib3)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['QISKIT_SETTINGS'] = '{"suppress_warnings": true}'

logging.getLogger('qiskit').setLevel(logging.CRITICAL)
logging.getLogger('stevedore').setLevel(logging.CRITICAL)
logging.getLogger('urllib3').setLevel(logging.CRITICAL)
logging.getLogger('qiskit_ibm_runtime').setLevel(logging.CRITICAL)

from qiskit_ibm_runtime import QiskitRuntimeService
import os

# Load saved account
try:
    service = QiskitRuntimeService()
    print(f"Connected to IBM Quantum: {service.channel}")
except:
    print("No saved account found.")
    token = input("Please enter your IBM Quantum API Token: ")
    service = QiskitRuntimeService(channel="ibm_quantum", token=token)

# Hardware Selection Strategy
# We prioritize the 156-qubit 'ibm_marrakesh' or 'ibm_fez' for topological fidelity.
desired_backends = ["ibm_marrakesh", "ibm_fez", "ibm_torino"]
backend = None

for name in desired_backends:
    try:
        backend = service.backend(name)
        print(f"Targeting High-Fidelity Hardware: {backend.name}")
        break
    except:
        continue

if backend is None:
    # Fallback to whatever is available
    backend = service.least_busy(operational=True, simulator=False, min_num_qubits=127)
    print(f"Fallback Target: {backend.name}")

Connected to IBM Quantum: ibm_quantum_platform
Targeting High-Fidelity Hardware: ibm_marrakesh


In [3]:
# --- QUANTUM LOOM: PRIMORDIAL B-MODE SIGNATURE ---
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
import numpy as np

# 2. DEFINE THE PHYSICS (SCALAR VS TENSOR)
# We compare Standard Inflation (Scalar) vs Quantum Loom Freezing (Tensor).

circuits = []
modes = ["SCALAR (Standard)", "TENSOR (Quantum Loom)"]

for mode in modes:
    # We use a 4-qubit Plaquette (0,1 top row; 2,3 bottom row)
    # 0 -- 1
    # |    |
    # 2 -- 3
    qr = QuantumRegister(4, 'cmb')
    cr = ClassicalRegister(4, 'meas')
    qc = QuantumCircuit(qr, cr)
    
    # A. INITIAL STATE (The Hot Superfluid)
    # All nodes in superposition
    qc.h(qr)
    
    if mode == "SCALAR (Standard)":
        # Scalar Mode: Density Pulse (E-Mode)
        # All qubits get the same phase kick. No "Twist".
        # This represents standard energy density fluctuations.
        qc.rz(np.pi/4, qr) 
        
    else: # TENSOR (Quantum Loom)
        # Tensor Mode: Topological Rupture (B-Mode)
        # We simulate the "Freezing" by entangling diagonals with OPPOSITE phase.
        # This creates a "Shear" or "Curl" in the geometry.
        
        # Diagonal 0-3 (Positive Twist)
        qc.rzz(np.pi/2, 0, 3)
        # Diagonal 1-2 (Negative Twist)
        qc.rzz(-np.pi/2, 1, 2)
        
        # This "Quadrupole" distortion is the signature of a Gravitational Wave.

    # B. MEASUREMENT (Polarization Analysis)
    # To detect B-modes (Curl), we need to measure in a basis sensitive to Phase Chirality.
    # Standard Z-measure sees Density. 
    # Bell-measure sees Entanglement.
    # We use a "Wilson Loop" style measurement:
    # We project the "Curl" onto the measurement basis.
    
    # 1. Apply CNOTs around the ring to capture the total flux/twist
    qc.cx(0, 1)
    qc.cx(1, 3)
    qc.cx(3, 2)
    qc.cx(2, 0)
    
    # 2. Measure in X-basis (Hadamard + Measure) to see the Phase interference
    qc.h(qr)
    qc.measure(qr, cr)
    
    circuits.append(qc)

# 3. TRANSPILE & RUN
print(f"Simulating Primordial Phase Transitions...")
isa_circuits = transpile(circuits, backend=backend)
sampler = SamplerV2(mode=backend)
job = sampler.run(isa_circuits, shots=4096)
print(f"Job ID: {job.job_id()}")

result = job.result()

# 4. ANALYZE: DETECTING THE SIGNAL
print("\n=== PRIMORDIAL B-MODE REPORT ===")
print(f"{'THEORY MODEL':<25} | {'B-MODE SIGNAL (Twist)':<20} | {'VERDICT'}")
print("-" * 70)

for i, mode in enumerate(modes):
    counts = result[i].data.meas.get_counts()
    total = sum(counts.values())
    
    # Analysis Logic:
    # In our "Wilson Loop" setup:
    # - E-Modes (Scalar) tend to cancel out in the loop or sum constructively to |0000> / |1111>.
    # - B-Modes (Tensor/Twist) create "Odd Parity" states (e.g., |0101>, |1010>) due to the shear.
    
    odd_parity_count = 0
    for bitstring, count in counts.items():
        # Check parity (Hamming weight)
        if bitstring.count('1') % 2 != 0:
            odd_parity_count += count
            
    # B-Mode Signal Strength = Odd Parity Fraction
    # (Simplified proxy for Curl measurement)
    b_signal = odd_parity_count / total
    
    verdict = "SCALAR DOMINANT"
    if b_signal > 0.4: verdict = "TENSOR DETECTED (B-MODE)"
    
    print(f"{mode:<25} | {b_signal*100:.1f}%                | {verdict}")

print("-" * 70)

Simulating Primordial Phase Transitions...
Job ID: d686erre4kfs73d2njkg

=== PRIMORDIAL B-MODE REPORT ===
THEORY MODEL              | B-MODE SIGNAL (Twist) | VERDICT
----------------------------------------------------------------------
SCALAR (Standard)         | 17.5%                | SCALAR DOMINANT
TENSOR (Quantum Loom)     | 52.1%                | TENSOR DETECTED (B-MODE)
----------------------------------------------------------------------
